In [8]:
import numpy as np
import json
import math
import random

In [9]:
transition_freq_map = {
    (2, 1): 121.57,
    (3, 1): 102.57,
    (4, 1): 97.254,
    (5, 1): 94.974,
    (6, 1): 93.780,
    (7, 1): 93.075,
    (3, 2): 656.279,
    (4, 2): 486.135,
    (5, 2): 434.047,
    (6, 2): 410.173,
    (7, 2): 397.008,
    (4, 3): 1875.,
    (5, 3): 1282.,
    (6, 3): 1094.,
    (7, 3): 1005.,
    (5, 4): 4051.,
    (6, 4): 2625.,
    (7, 4): 2166.,
    (6, 5): 7460.,
    (7, 5): 4654.,
    (7, 6): 12370.
}


possible_emwidths = ["normal", "doubly-ext", "UV-ext", "IR-ext", "UV-dim", "IR-dim"]
possible_emshapes = ["none", "tri", "tri-inv", "lowpass", "hipass", "bandpass"]
possible_soundshapes = ["up", "down", "spike", "free"]

possible_soundtrans = [(5, 2), (5, 3), (6, 3), (7, 3), (7, 4), (7, 5)]
possible_haptrans = [(3, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2),
 (4, 3),
 (5, 3),
 (6, 3),
 (7, 3),
 (5, 4),
 (6, 4),
 (7, 4),
 (6, 5),
 (7, 5),
 (7, 6)]


class Transition:
    
    outer = None
    inner = None
    freq = None 

    def __init__(self, outer: int, inner: int):
        self.outer = outer
        self.inner = inner
        self.freq = self.get_freq()
        
    def get_freq(self):
        return transition_freq_map[(self.outer, self.inner)]
    
    def to_dict(self):
        return {"outer": self.outer, "inner": self.inner, "emfreq": self.freq }
        
    def __expr__(self):
        return self.outer+ " -> "+ self.inner
        
    def __repr__(self):
        return "("+ str(self.outer) + " -> "+ str(self.inner) + ")"

    
    
class Emission:
    
    transition = None
    width = None
    
    def __init__(self, transition: Transition, width: str):
        self.transition = transition
        self.width = width 
        
    def to_dict(self):
        return self.transition.to_dict() | {"width": self.width}
    
    
    
    
class Sound:
    
    transition = None
    freq = None 
    
    def __init__(self, transition: Transition, exp):
        self.transition = transition
        self.freq = self.get_freq(exp)
        
    def get_freq(self, exp):
        return 6 ** exp * math.log(self.transition.freq, 6)
        
    def to_dict(self):
        return self.transition.to_dict() | {"soundfreq": self.freq}
    
    
class Haptics:
    
    transition = None
    
    def __init__(self, transition: Transition):
        self.transition = transition
    
    def to_dict(self):
        return {"outer": self.transition.outer, "inner": self.transition.inner}
    
    
    
    
    
class EMphoneme:
    
    E = None
    S = None
    H = None
   
    def __init__(self, E: Emission, S: list[Sound], H: Haptics):
        self.E = E
        self.S = S
        self.H = H
        
    def to_dict(self):
        return {"emission": self.E.to_dict(), 
                "sounds": [x.to_dict() for x in self.S], 
                "haptics": self.H.to_dict()}

    
    
    
class EMmorpheme:
    
    phoneme = None
    emshape = None
    soundshape = None
    
    def __init__(self, phoneme: EMphoneme, emshape: str, soundshape: str):
        self.phoneme = phoneme
        self.emshape = emshape
        self.soundshape = soundshape
        
    def to_dict(self):
        return {"phoneme": self.phoneme.to_dict(), 
                "emshape": self.emshape, 
                "soundshape": self.soundshape}
    
    
class EMword:
    
    morphemes = []
    
    def __init__(self, morphemes: list[EMmorpheme]):
        self.morphemes = morphemes
    
    def to_dict(self):
        return {"m1": self.morphemes[0].to_dict(),
                "m2": self.morphemes[1].to_dict(), 
                "m3": self.morphemes[2].to_dict(),
                "m4": self.morphemes[3].to_dict(), 
                "m5": self.morphemes[4].to_dict(),
                "m6": self.morphemes[5].to_dict()}
    
    def save_json(self, filename):
        dict = self.to_dict()
        with open (filename, "w") as file:
            json.dump(dict, file, indent=4)
            
            
class EMsentence:
    
    words = []
    
    def __init__(self, words: list[EMword]):
        self.words = words
        
    def to_dict(self):
        return [x.to_dict() for x in self.words]
    
    def save_json(self, filename):
        dict = self.to_dict()
        with open(filename, "w") as file:
            json.dump(dict, file, indent = 4)

In [10]:
def random_morpheme(exp=None):
    p = random_phoneme(exp)
    emshape = random.choice(possible_emshapes)
    soundshapes = random_soundshapes(p)
    
    rm = EMmorpheme(p, emshape, soundshapes)
    return rm

def random_phoneme(exp ):
    emission = random_emission()
    sounds = random_sounds(exp)
    haptics = random_haptics()
    
    rp = EMphoneme(emission, sounds, haptics)
    return rp 

def random_emission():
    t = random.choice(list(transition_freq_map.keys()))
    trans = Transition(t[0], t[1])
    width = random.choice(possible_emwidths)
    
    re = Emission(trans, width)
    return re

def random_sounds(exp):
    num_sounds = random.choice([1, 2, 3])
    trans = []
    while len(trans) != num_sounds:
        t = random.choice(possible_soundtrans)
        if t not in trans: 
            trans.append(t)
            
    sounds = []
    for tran in trans:
        rs = Sound(Transition(tran[0], tran[1]), exp)
        sounds.append(rs)
        
    return sounds

def random_haptics():
    t = random.choice(list(possible_haptrans))
    trans = Transition(t[0], t[1])
    rh = Haptics(trans)
    return rh

def random_soundshapes(p):
    soundshapes = []
    while len(soundshapes)!= len(p.S):
        choice = random.choice(possible_soundshapes)
        if(choice == "free") and "down" not in soundshapes: 
            soundshapes.append(random.choice(possible_soundshapes))
        elif(choice not in soundshapes):
            soundshapes.append(random.choice(possible_soundshapes))
        
    return soundshapes


In [11]:
def random_word(exp=None):
    morphemes = []
    for i in range(6):
        morphemes.append(random_morpheme(exp=exp))
    
    rw = EMword(morphemes)
    return rw

In [12]:
def random_sentence(exp = None):
    words = []
    for i in range(21):
        words.append(random_word(exp=exp))
    
    rs = EMsentence(words)
    
    return rs

In [39]:
sentence = random_sentence(exp = 3)

In [40]:
sentence.save_json("random_sentence_16.json")

In [14]:
word = random_word(exp = 3)

In [15]:
word.save_json("random_word.json")

In [2]:
possible_soundtrans.append((4, 2))
possible_soundtrans.append((6, 2))
possible_soundtrans.append((6, 4))
for sound in possible_soundtrans:
    print(sound, 6 ** 3 * math.log(transition_freq_map[sound], 6))

NameError: name 'possible_soundtrans' is not defined